In [114]:
## what effect does time have on location, district, race, age 

In [115]:
import pandas as pd
from lib import standardize_item_numbers

def clean():
    dfa = pd.read_csv("../data/electronic_police_report_2018_2022.csv")
    dfa = dfa.pipe(standardize_item_numbers, ["item_number"])

    dfb = pd.read_csv("../data/rtcc.csv")
    dfb["footage_requested"] = "1"
    dfb = dfb.pipe(standardize_item_numbers, ["item_number"])

    df = pd.merge(dfa, dfb, on="item_number")
    return df

In [116]:
df = clean()

C:\Users\PC\AppData\Local\Temp\ipykernel_22428\1662552057.py:11: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df4 = pd.read_csv("../data/electronic_police_report_2021.csv")
C:\Users\PC\AppData\Local\Temp\ipykernel_22428\1662552057.py:12: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df5 = pd.read_csv("../data/electronic_police_report_2022.csv")


In [117]:
df.item_number.unique()

array(['d2077118', 'e1274818', 'l2194418', ..., 'j2225922', 'a1349122',
       'a3158222'], dtype=object)

In [118]:
df = df.drop_duplicates(subset=["offender_race", "offender_gender", "offender_age", "item_number"])

In [119]:
datetimes = (df.occurred_date_time.str.replace(r"(\w{4})-(\w{1,2})-(\w{1,2})", r"\2/\3/\1").str.extract(r"(\w{2}):(\w{2}):?(\w{2})?"))
df.loc[:, "occurred_datetime"] = datetimes[0]
df.occurred_datetime.value_counts()

C:\Users\PC\AppData\Local\Temp\ipykernel_22428\2488137716.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  datetimes = (df.occurred_date_time.str.replace(r"(\w{4})-(\w{1,2})-(\w{1,2})", r"\2/\3/\1")


23    474
21    444
20    430
19    414
01    412
17    401
18    400
22    387
16    368
02    362
00    359
15    358
03    333
11    321
14    314
12    291
10    273
04    255
13    250
05    158
09    153
08    146
06    138
07    119
Name: occurred_datetime, dtype: int64

In [120]:
df.location.count().sum()

8218

In [121]:
df.shape

(8218, 28)

In [122]:
requested = df[df.footage_requested.isin(["1"])]

In [123]:
requested.year.value_counts()

2021    2043
2020    1940
2022    1715
2019    1531
2018     989
Name: year, dtype: int64

In [124]:
requested.groupby("year").disposition.value_counts()

year  disposition
2018  OPEN            603
      CLOSED          365
      UNFOUNDED        21
2019  OPEN            932
      CLOSED          569
      UNFOUNDED        30
2020  OPEN           1415
      CLOSED          489
      UNFOUNDED        36
2021  OPEN           1527
      CLOSED          465
      UNFOUNDED        51
2022  OPEN           1264
      CLOSED          429
      UNFOUNDED        22
Name: disposition, dtype: int64

In [125]:
requested.groupby("year").disposition.value_counts(normalize=True)

year  disposition
2018  OPEN           0.609707
      CLOSED         0.369060
      UNFOUNDED      0.021234
2019  OPEN           0.608752
      CLOSED         0.371653
      UNFOUNDED      0.019595
2020  OPEN           0.729381
      CLOSED         0.252062
      UNFOUNDED      0.018557
2021  OPEN           0.747430
      CLOSED         0.227606
      UNFOUNDED      0.024963
2022  OPEN           0.737026
      CLOSED         0.250146
      UNFOUNDED      0.012828
Name: disposition, dtype: float64

In [126]:
# r_ct = requested.groupby("year").disposition.value_counts().to_excel("../data/excel/requested_by_year_ct.xlsx")

In [127]:
# r_pct = requested.groupby("year").disposition.value_counts(normalize=True).to_excel("../data/excel/requested_by_year_pct.xlsx")

In [128]:
not_requested = df[~((df.footage_requested == "1.0"))]

In [129]:
not_requested.groupby("year").disposition.value_counts()

year  disposition
2018  OPEN            603
      CLOSED          365
      UNFOUNDED        21
2019  OPEN            932
      CLOSED          569
      UNFOUNDED        30
2020  OPEN           1415
      CLOSED          489
      UNFOUNDED        36
2021  OPEN           1527
      CLOSED          465
      UNFOUNDED        51
2022  OPEN           1264
      CLOSED          429
      UNFOUNDED        22
Name: disposition, dtype: int64

In [130]:
not_requested.groupby("year").disposition.value_counts(normalize=True)

year  disposition
2018  OPEN           0.609707
      CLOSED         0.369060
      UNFOUNDED      0.021234
2019  OPEN           0.608752
      CLOSED         0.371653
      UNFOUNDED      0.019595
2020  OPEN           0.729381
      CLOSED         0.252062
      UNFOUNDED      0.018557
2021  OPEN           0.747430
      CLOSED         0.227606
      UNFOUNDED      0.024963
2022  OPEN           0.737026
      CLOSED         0.250146
      UNFOUNDED      0.012828
Name: disposition, dtype: float64

In [131]:
r_ct = pd.DataFrame(not_requested.groupby("year").disposition.value_counts())
r_ct  = r_ct.rename(columns={r_ct.columns[0]: "requested_ct"})
r_ct = r_ct.reset_index()

r_pct = pd.DataFrame(not_requested.groupby("year").disposition.value_counts(normalize=True))
r_pct  = r_pct.rename(columns={r_pct.columns[0]: "requested_pct"})
r_pct = r_pct.reset_index()

nr_ct = pd.DataFrame(not_requested.groupby("year").disposition.value_counts())
nr_ct  = nr_ct.rename(columns={nr_ct.columns[0]: "not_requested_ct"})
nr_ct = nr_ct.reset_index()

nr_pct = pd.DataFrame(not_requested.groupby("year").disposition.value_counts(normalize=True))
nr_pct  = nr_pct.rename(columns={nr_pct.columns[0]: "not_requested_pct"})
nr_pct = nr_pct.reset_index()


In [132]:
dfs = pd.concat([nr_pct, nr_ct, r_pct, r_ct])

In [133]:
dfs

,year,disposition,not_requested_pct,not_requested_ct,requested_pct,requested_ct
0,2018,OPEN,0.609707,NaN,NaN,NaN
1,2018,CLOSED,0.369060,NaN,NaN,NaN
2,2018,UNFOUNDED,0.021234,NaN,NaN,NaN
3,2019,OPEN,0.608752,NaN,NaN,NaN
4,2019,CLOSED,0.371653,NaN,NaN,NaN
5,2019,UNFOUNDED,0.019595,NaN,NaN,NaN
6,2020,OPEN,0.729381,NaN,NaN,NaN
7,2020,CLOSED,0.252062,NaN,NaN,NaN
8,2020,UNFOUNDED,0.018557,NaN,NaN,NaN
9,2021,OPEN,0.747430,NaN,NaN,NaN


In [134]:
# df.to_excel("../data/requests_per_year_2018_present.xlsx")